<style>
    .titulo-violeta {
        color: #c485ff;
    }
</style>

<h1 class="titulo-violeta">TP Convolutional Neural Networks</h1>

## <span style="color:lightblue">Avanzado IA</span>

### <span style="color:lightblue">Tp hecho por Benjamin Kapustiansky, Gabriel Slotnisky, Lucas Bertoloni y Oliver Safdieh</span> 

## Problematica a resolver:
Un programa no podia diferenciar entre algunos tipos de frutas (apple, palta, banana, cereza y kiwi)

## Solución propuesta:
Mediante un modelo de CNN, nosotros logramos identificar y diferenciar estas frutas. Es una clasificación multiclase de una sola salida. En este caso, las clases son 5.

## <span style="color:lightblue">Libraries</span>

In [1]:
# Estas líneas de código nos permiten instalar opencv desde Jupyter Notebook, que vamos a necesitar para importar 'cv2'
import sys
!{sys.executable} -m pip install opencv-python --user


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install matplotlib
%pip install numpy
%pip install tensorflow
%pip install pandas
%pip install scikit-learn
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# import tensorflow as tf
# from tensorflow import keras


# print(tf.__version__)

In [7]:
import seaborn as sns

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import csv

from PIL import Image

import os
import shutil

import random
SEED = 456
random.seed(SEED)

## <span style="color:lightblue">Dataset</span>

### Introducción

El dataset lo armamos nosotros con imagenes extraidas de Kaggle. Estas imagenes estaban ya distribuidas en capertas distintas entre train y test y subdivididas en sus respectivas clases. Lo que nostros hicimos a mano, fue filtrar imagenes que no nos servian y ademas unificar estas imagenes por clase para luego nosotros decidir la proporcion a utilizar entre TRAIN, TEST y VALIDATION

Link al [Kaggle](https://www.kaggle.com/datasets/karimabdulnabi/fruit-classification10-class?select=MY_data)

### Armado del Dataset

Mover imagenes desde la carpeta original (Frutas) hacia la nueva creada por nosotros (Fruits)

In [5]:
# Definir las rutas de las carpetas
apple_paths = ["Frutas/apple", "Frutas/apple2"]
avocado_paths = ["Frutas/avocado", "Frutas/avocado2"]
banana_paths = ["Frutas/banana", "Frutas/banana2"]
cherry_paths = ["Frutas/cherry", "Frutas/cherry2"]
kiwi_paths = ["Frutas/kiwi", "Frutas/kiwi2"]

# Función para listar imágenes en carpetas
def list_images_in_paths(paths):
    images = []
    for path in paths:
        if os.path.exists(path):  # Verificar si la carpeta existe
            images.extend([os.path.join(path, file) for file in os.listdir(path) if file.endswith((".jpg", ".png", ".jpeg"))])
    return images

# Crear listas de imágenes divididas por clase
apple_images = list_images_in_paths(apple_paths)
avocado_images = list_images_in_paths(avocado_paths)
banana_images = list_images_in_paths(banana_paths)
cherry_images = list_images_in_paths(cherry_paths)
kiwi_images = list_images_in_paths(kiwi_paths)

# Ruta principal "fruits"
fruits_dir = "fruits"

# Crear carpeta principal si no existe
if not os.path.exists(fruits_dir):
    os.makedirs(fruits_dir)

# Función para mover las imágenes a las subcarpetas correspondientes
def move_images_to_fruit_folder(fruit_name, image_list):
    # Crear la subcarpeta de la fruta si no existe
    fruit_folder = os.path.join(fruits_dir, fruit_name)
    if not os.path.exists(fruit_folder):
        os.makedirs(fruit_folder)
    
    # Mover cada imagen a su carpeta correspondiente
    for image_path in image_list:
        if os.path.exists(image_path):
            # Obtener el nombre de la imagen
            image_name = os.path.basename(image_path)
            # Definir la ruta de destino
            destination = os.path.join(fruit_folder, image_name)
            # Mover la imagen
            shutil.move(image_path, destination)
            print(f"Moved {image_name} to {fruit_folder}")

# Mover las imágenes de cada clase a sus respectivas carpetas
move_images_to_fruit_folder("apple", apple_images)
move_images_to_fruit_folder("avocado", avocado_images)
move_images_to_fruit_folder("banana", banana_images)
move_images_to_fruit_folder("cherry", cherry_images)
move_images_to_fruit_folder("kiwi", kiwi_images)


Aca pasamos a crear el csv con las imagenes que estan en la carpeta Fruits

In [8]:
# Definir las listas de imágenes y sus etiquetas
classes = {
    0: "apple",
    1: "avocado",
    2: "banana",
    3: "cherry",
    4: "kiwi",
}

# Función para generar una lista de imágenes y etiquetas
def generate_image_label_list():
    image_label_list = []
    for label, fruit in classes.items():
        fruit_folder = os.path.join(fruits_dir, fruit)
        if os.path.exists(fruit_folder):
            # Listar todas las imágenes de la carpeta
            images = [os.path.join(fruit_folder, file) for file in os.listdir(fruit_folder) if file.endswith((".jpg", ".png", ".jpeg"))]
            # Añadir las imágenes con su etiqueta
            image_label_list.extend([[image, label] for image in images])
    return image_label_list

# Generar la lista de imágenes y etiquetas
image_label_list = generate_image_label_list()

# Crear el archivo CSV
csv_file_path = "fruits_dataset.csv"
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # Escribir los encabezados
    writer.writerow(["Image", "Label"])
    # Escribir las filas (imagen, etiqueta)
    writer.writerows(image_label_list)

print(f"CSV file created successfully: {csv_file_path}")


CSV file created successfully: fruits_dataset.csv


### Import dataset

In [9]:
archivo = "fruits.csv"

# Leer datos del archivo
frutas : pd.DataFrame = pd.read_csv(archivo)
frutas

,Image,Label
0,fruits\apple\images.jpeg,0
1,fruits\apple\img_01.jpeg,0
2,fruits\apple\img_1001.jpeg,0
3,fruits\apple\img_101.jpeg,0
4,fruits\apple\img_1011.jpeg,0
...,...,...
1596,fruits\kiwi\img_931.jpeg,4
1597,fruits\kiwi\img_941.jpeg,4
1598,fruits\kiwi\img_961.jpeg,4
1599,fruits\kiwi\img_971.jpeg,4


### Dataset Split

## <span style="color:lightblue">Create CNN Model</span>